In [531]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, ThresholdedReLU, MaxPooling2D, Embedding, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import gc

In [532]:
df=pd.read_excel("datathon.xlsx",skiprows=3)

In [536]:
df.head(10)

,Cluster,Brand Group,Country,Function,Jan 2012,Feb 2012,Mar 2012,Apr 2012,May 2012,Jun 2012,...,Mar 2018,Apr 2018,May 2018,Jun 2018,Jul 2018,Aug 2018,Sep 2018,Oct 2018,Nov 2018,Dec 2018
0,Cluster 3,Brand Group 12,Country 19,Sales 1,1861.328751,1995.945017,1998.533792,2019.243986,2425.681558,2443.802978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cluster 3,Brand Group 12,Country 19,Sales 2,1316.247027,1450.223283,1440.837462,1564.066421,2014.132904,1792.307471,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cluster 3,Brand Group 12,Country 19,Investment 1,-216.162658,-238.167239,-217.457045,-214.868270,-390.904926,-120.378007,...,-159.727377,-141.994273,-127.367698,-162.445590,-146.395189,-145.100802,-171.894616,-136.946163,-189.974966,-156.680908
3,Cluster 3,Brand Group 12,Country 19,Investment 2,-107.434135,-14.238259,-62.130584,-91.901489,-199.335624,-253.699885,...,-43.361970,-83.487973,-34.560137,-37.407789,-35.595647,-64.460481,-111.705613,-29.253150,-133.434491,-88.018327
4,Cluster 3,Brand Group 12,Country 19,Investment 4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.682703,0.000000,0.000000,0.000000,0.000000,0.000000,-3.106529,0.000000,0.000000,0.000000
5,Cluster 3,Brand Group 12,Country 19,Investment 6,-56.953036,-55.658648,-64.719359,-56.953036,-55.658648,-93.195876,...,-47.115693,-49.445590,-44.268041,-56.176403,-42.197022,-41.032073,-48.539519,-45.044674,-45.007720,-35.730998
6,Cluster 3,Brand Group 17,Country 19,Sales 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cluster 3,Brand Group 17,Country 19,Sales 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Cluster 3,Brand Group 17,Country 19,Investment 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-2105.967927,-1952.841924,-2053.674685,-2294.171821,-2123.312715,-1912.457045,-2342.970218,-2114.899198,-2206.086863,-1968.091008
9,Cluster 3,Brand Group 17,Country 19,Investment 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-2193.339061,-1045.735395,-3997.844215,-2019.891180,-1790.266896,-1048.194731,-1904.690722,-1511.585338,-1595.179450,-1041.791168


##  Sales1

In [537]:
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
dfsales2=df.loc[df['Function'] == "Sales 2"]
dfsales2 = dfsales2[dfsales2.columns.drop(list(dfsales2.filter(regex='2018')))]
dfsales2=dfsales2.groupby(['Cluster','Brand Group']).agg('sum')
dfmelt=pd.melt(dfsales2.reset_index(), id_vars=["Cluster", "Brand Group"], 
                  var_name="Date", value_name="Value_Y")

In [677]:
dfmelt.head(5)

,Cluster,Brand Group,Date,Value_Y
0,Cluster 1,Brand Group 12,Jan 2012,616.408961
1,Cluster 1,Brand Group 13,Jan 2012,62.381470
2,Cluster 1,Brand Group 15,Jan 2012,0.000000
3,Cluster 1,Brand Group 16,Jan 2012,2457.695992
4,Cluster 1,Brand Group 17,Jan 2012,0.000000


## Sales 2

In [555]:
dfinves=df.loc[df['Function'].str.contains("Investment")]

In [556]:
dfinves=dfinves.groupby(['Cluster','Brand Group','Function']).agg('sum')


In [558]:
dfinves=pd.melt(dfinves.reset_index(), id_vars=["Cluster", "Brand Group","Function"], 
                  var_name="Date", value_name="Investment")

In [567]:
dfinves=dfinves.pivot_table('Investment', ['Cluster', 'Brand Group','Date'], 'Function')

In [681]:
dfinves=dfinves.reset_index()

In [691]:
dfinves=dfinves.fillna(0)

In [699]:
dfinves.head(5)

,Cluster,Brand Group,Date_coded,Investment 1,Investment 2,Investment 3,Investment 4,Investment 5,Investment 6
0,Cluster 1,Brand Group 12,3,-39.664141,-79.814354,0.000000,-1.451180,0.0,-9.745249
1,Cluster 1,Brand Group 12,15,-44.510635,-144.391369,0.000000,-1.462129,0.0,-12.250295
2,Cluster 1,Brand Group 12,27,-51.325172,-118.808299,-0.682549,-3.965413,0.0,-11.078440
3,Cluster 1,Brand Group 12,39,-47.388726,-62.490278,0.000000,-3.320586,0.0,-14.386244
4,Cluster 1,Brand Group 12,51,-55.299492,-115.182981,0.000000,-8.180448,0.0,-7.625617


In [694]:
dfinves=dfinves.replace({"Date": dictmonths})

In [698]:
dfinves.columns=['Cluster', 'Brand Group', 'Date_coded', 'Investment 1', 'Investment 2',
       'Investment 3', 'Investment 4', 'Investment 5', 'Investment 6']

In [ ]:
dfmerged=dfmerged.fillna(0)

In [682]:
dfmerged=pd.merge(dfmelt,dfinves,
                 on=['Cluster','Brand Group','Date'])

In [683]:
dfmerged.head(5)

,Cluster,Brand Group,Date,Value_Y,Investment 1,Investment 2,Investment 3,Investment 4,Investment 5,Investment 6
0,Cluster 1,Brand Group 12,Jan 2012,616.408961,-35.2948,-31.457517,0.0,-0.286406,NaN,-7.976037
1,Cluster 1,Brand Group 15,Jan 2012,0.000000,0.0000,0.000000,NaN,0.000000,NaN,NaN
2,Cluster 1,Brand Group 16,Jan 2012,2457.695992,NaN,0.024731,NaN,-1.545383,NaN,NaN
3,Cluster 1,Brand Group 17,Jan 2012,0.000000,0.0000,0.000000,0.0,0.000000,NaN,0.000000
4,Cluster 1,Brand Group 18,Jan 2012,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN


In [527]:
dictmonths=dict(zip(dfmelt["Date"].unique(),range(len(dfmelt["Date"].unique()))))

In [529]:
dfmelt["Date_coded"]=dfmelt["Date"].apply(lambda x: dictmonths[x])

In [388]:
dfmelt["Month"]=dfmelt["Date_coded"].apply(lambda x: x%12)

In [393]:
dfmelt.head(5)

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Actual,Lag-1,Lag-2,Lag-3
0,Cluster 1,Brand Group 12,Jan 2012,616.408961,0,0,NaN,NaN,NaN,NaN
1,Cluster 1,Brand Group 13,Jan 2012,62.381470,0,0,NaN,NaN,NaN,NaN
2,Cluster 1,Brand Group 15,Jan 2012,0.000000,0,0,NaN,NaN,NaN,NaN
3,Cluster 1,Brand Group 16,Jan 2012,2457.695992,0,0,NaN,NaN,NaN,NaN
4,Cluster 1,Brand Group 17,Jan 2012,0.000000,0,0,NaN,NaN,NaN,NaN


In [390]:
import numpy as np
dfmelt["Actual"] = np.nan
dfmelt["Lag-1"] = np.nan
dfmelt["Lag-2"] = np.nan
dfmelt["Lag-3"] = np.nan
# dfmelt["Lag-4"] = np.nan

In [391]:
  &(dfpre["Date_coded"]==dfpre.iloc[i].Date_coded-lag)]["Value"].values[0]

SyntaxError: invalid syntax (<ipython-input-391-14710708d19c>, line 1)

In [369]:
a=dfmelt[(dfmelt["Cluster"]==dfmelt.iloc[i].Cluster) 
           & (dfmelt["Brand Group"]==dfmelt.iloc[i,dfmelt.columns.get_loc("Brand Group")])]

In [392]:
dfpre[(dfpre["Cluster"]==dfpre.iloc[i].Cluster) 
                   & (dfpre["Brand Group"]==dfpre.iloc[i,dfpre.columns.get_loc("Brand Group")])]

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Lag-1,Lag-2,Lag-3,Lag-6,Lag-12,Actual
0,Cluster 1,Brand Group 12,Jan 2012,616.408961,0,0,NaN,NaN,NaN,NaN,NaN,NaN
303,Cluster 1,Brand Group 12,Feb 2012,685.081035,1,1,NaN,NaN,NaN,NaN,NaN,NaN
606,Cluster 1,Brand Group 12,Mar 2012,968.938870,2,2,NaN,NaN,NaN,NaN,NaN,NaN
909,Cluster 1,Brand Group 12,Apr 2012,723.701936,3,3,NaN,NaN,NaN,NaN,NaN,NaN
1212,Cluster 1,Brand Group 12,May 2012,656.607506,4,4,NaN,NaN,NaN,NaN,NaN,NaN
1515,Cluster 1,Brand Group 12,Jun 2012,319.401731,5,5,NaN,NaN,NaN,NaN,NaN,NaN
1818,Cluster 1,Brand Group 12,Jul 2012,953.191022,6,6,NaN,NaN,NaN,NaN,NaN,NaN
2121,Cluster 1,Brand Group 12,Aug 2012,658.304347,7,7,NaN,NaN,NaN,NaN,NaN,NaN
2424,Cluster 1,Brand Group 12,Sep 2012,837.083837,8,8,NaN,NaN,NaN,NaN,NaN,NaN
2727,Cluster 1,Brand Group 12,Oct 2012,943.346418,9,9,NaN,NaN,NaN,NaN,NaN,NaN


In [394]:
def add_lag_values(dfpre,lag,lagname):
    for i in range (len (dfpre)):
        if dfpre.iloc[i].Date_coded>12:
            dfpre.iloc[i,dfpre.columns.get_loc(lagname)]=dfpre[(dfpre["Cluster"]==dfpre.iloc[i].Cluster) 
                   & (dfpre["Brand Group"]==dfpre.iloc[i,dfpre.columns.get_loc("Brand Group")])
                                                                &(dfpre["Date_coded"]==dfpre.iloc[i].Date_coded-lag)]["Value_Y"].values[0]
    return dfpre

In [395]:
dfpre=add_lag_values(dfmelt,1,'Actual')
dfpre=add_lag_values(dfmelt,2,'Lag-1')
dfpre=add_lag_values(dfpre,3,'Lag-2')
dfpre=add_lag_values(dfpre,4,'Lag-3')

In [399]:
dfpre['Month']=dfpre['Month'].apply(lambda x: 12 if x == 0 else x-1)

In [400]:
dfpre['sales_trend'] = dfpre['Actual']

lag_list=[1,2,3]

for lag in [1,2,3]:
    ft_name = ('Lag-%s' % lag)
    dfpre['sales_trend'] -= dfpre[ft_name]

dfpre['sales_trend'] /= len(lag_list) + 1

In [684]:
dfpre.head(5)

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Actual,Lag-1,Lag-2,Lag-3,sales_trend
0,Cluster 1,Brand Group 12,Jan 2012,616.408961,0,12,NaN,NaN,NaN,NaN,NaN
1,Cluster 1,Brand Group 13,Jan 2012,62.381470,0,12,NaN,NaN,NaN,NaN,NaN
2,Cluster 1,Brand Group 15,Jan 2012,0.000000,0,12,NaN,NaN,NaN,NaN,NaN
3,Cluster 1,Brand Group 16,Jan 2012,2457.695992,0,12,NaN,NaN,NaN,NaN,NaN
4,Cluster 1,Brand Group 17,Jan 2012,0.000000,0,12,NaN,NaN,NaN,NaN,NaN


In [687]:
dfpre_sales['Date_coded']=dfpre_sales['Date_coded'].apply(lambda x: x-1)

C:\Users\dsalvadu\AppData\Local\Continuum\anaconda3\envs\CDAC\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [702]:
dfinves.head(5)

,Cluster,Brand Group,Date_coded,Investment 1,Investment 2,Investment 3,Investment 4,Investment 5,Investment 6
0,Cluster 1,Brand Group 12,3,-39.664141,-79.814354,0.000000,-1.451180,0.0,-9.745249
1,Cluster 1,Brand Group 12,15,-44.510635,-144.391369,0.000000,-1.462129,0.0,-12.250295
2,Cluster 1,Brand Group 12,27,-51.325172,-118.808299,-0.682549,-3.965413,0.0,-11.078440
3,Cluster 1,Brand Group 12,39,-47.388726,-62.490278,0.000000,-3.320586,0.0,-14.386244
4,Cluster 1,Brand Group 12,51,-55.299492,-115.182981,0.000000,-8.180448,0.0,-7.625617


In [703]:
dffinal=pd.merge(dfpre_sales,dfinves,
                 on=['Cluster','Brand Group','Date_coded'])

In [704]:
dffinal.head(10)

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Actual,Lag-1,Lag-2,Lag-3,sales_trend,Investment 1,Investment 2,Investment 3,Investment 4,Investment 5,Investment 6
0,Cluster 1,Brand Group 12,Feb 2013,886.812584,12,0,745.704077,1084.703889,946.821916,943.346418,-557.292036,-41.558871,-14.368532,0.0,-0.224499,0.0,-9.895950
1,Cluster 1,Brand Group 15,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2,Cluster 1,Brand Group 16,Feb 2013,2297.069386,12,0,1965.631205,2106.791079,2956.718563,2495.493383,-1398.342955,0.000000,0.000000,0.0,-1.298518,0.0,0.000000
3,Cluster 1,Brand Group 17,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,-3.018683,0.000000,0.0,0.000000,0.0,-2.833370
4,Cluster 1,Brand Group 18,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
5,Cluster 1,Brand Group 19,Feb 2013,1498.933024,12,0,1256.505397,1204.286111,1735.955221,1139.976953,-705.928222,-0.876540,0.000000,0.0,-5.600064,0.0,-1.772652
6,Cluster 1,Brand Group 24,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-13.167373,0.0,0.000000,0.0,0.000000
7,Cluster 1,Brand Group 25,Feb 2013,2398.767022,12,0,2229.490821,2138.460934,2336.864699,3205.431354,-1362.816541,-32.372728,-0.095763,0.0,-5.871387,0.0,0.000000
8,Cluster 1,Brand Group 26,Feb 2013,2710.350144,12,0,2550.119528,1905.375056,1800.682066,2760.493348,-979.107736,-185.888295,-12.242750,0.0,-3.011869,0.0,-15.900721
9,Cluster 1,Brand Group 27,Feb 2013,977.035873,12,0,1095.882446,517.510205,873.167812,1026.741672,-330.384311,0.000000,-0.009205,0.0,1.102464,0.0,0.000000


In [731]:
dfpre.to_pickle("lags_invest.pkl")

In [709]:
yvalues =dffinal["Value_Y"]
xvalues = dffinal.drop(labels=['Value_Y'],axis=1)

In [710]:
from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
##sc.fit(xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])
#xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']] =sc.transform(xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])

In [711]:
X_train, X_valid, y_train, y_valid = train_test_split(xvalues, yvalues, test_size=0.2, shuffle=True)

In [712]:

xvalues.columns

Index(['Cluster', 'Brand Group', 'Date', 'Date_coded', 'Month', 'Actual',
       'Lag-1', 'Lag-2', 'Lag-3', 'sales_trend', 'Investment 1',
       'Investment 2', 'Investment 3', 'Investment 4', 'Investment 5',
       'Investment 6'],
      dtype='object')

In [713]:
X_train=X_train[['Month','Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend','Investment 1',
       'Investment 2', 'Investment 3', 'Investment 4', 'Investment 5',
       'Investment 6']]
X_valid=X_valid[['Month','Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend','Investment 1',
       'Investment 2', 'Investment 3', 'Investment 4', 'Investment 5',
       'Investment 6']]

In [714]:
from xgboost import XGBRegressor

In [730]:
model = XGBRegressor(
    max_depth=5,
    n_estimators=1000,
    min_child_weight=10, 
    colsample_bytree=0.5, 
    subsample=0.9, 
    eta=0.75,    
    seed=42)

model.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_valid, y_valid)], 
    verbose=True, 
    early_stopping_rounds = 30)

[0]	validation_0-rmse:4380.77	validation_1-rmse:4517.38
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 30 rounds.
[1]	validation_0-rmse:3965.48	validation_1-rmse:4103.23
[2]	validation_0-rmse:3591.02	validation_1-rmse:3727.66
[3]	validation_0-rmse:3254.55	validation_1-rmse:3390.52
[4]	validation_0-rmse:2953.33	validation_1-rmse:3088.27
[5]	validation_0-rmse:2683.29	validation_1-rmse:2820.23
[6]	validation_0-rmse:2439.36	validation_1-rmse:2575.11
[7]	validation_0-rmse:2222.87	validation_1-rmse:2360.2
[8]	validation_0-rmse:2027.87	validation_1-rmse:2164.83
[9]	validation_0-rmse:1855.3	validation_1-rmse:1992.8
[10]	validation_0-rmse:1700.61	validation_1-rmse:1838.97
[11]	validation_0-rmse:1562.87	validation_1-rmse:1704.9
[12]	validation_0-rmse:1440.99	validation_1-rmse:1583.02
[13]	validation_0-rmse:1330.9	validation_1-rmse:1471.67
[14]	validation_0-rmse:1235.16	validation_1-rmse:1373.85
[

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, eta=0.75, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=10, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=42, silent=True, subsample=0.9)

# Pred dataset

In [719]:
dfsales2=dfsales2.groupby(['Cluster','Brand Group']).agg('sum')

In [720]:
dfsales2

Jan 2012      Feb 2012      Mar 2012  \
Cluster   Brand Group                                                
Cluster 1 Brand Group 12    616.408961    685.081035    968.938870   
          Brand Group 13     62.381470     69.445132     42.081080   
          Brand Group 15      0.000000      0.000000      0.000000   
          Brand Group 16   2457.695992   2616.171336   2019.736343   
          Brand Group 17      0.000000      0.000000      0.000000   
          Brand Group 18      0.000000      0.000000      0.000000   
          Brand Group 19   1328.072408   1747.087079   1843.272270   
          Brand Group 24      0.000000      0.000000      0.000000   
          Brand Group 25   1869.355444   1724.577575   3067.466622   
          Brand Group 26   2044.415276   2124.060822   2773.263487   
          Brand Group 27    616.696052    943.158430    991.609400   
          Brand Group 28      0.000000      0.000000      0.000000   
          Brand Group 30   1451.344742   2421.703682   2174.680037   
          Brand Group 31    601.508882   1221.516101    330.240589   
          Brand Group 36    224.091013     10.742677      3.547430   
          Brand Group 38    158.842381    140.429788    136.151464   
          Brand Group 39    667.435844    696.507436    618.169416   
          Brand Group 40    203.767645    186.701557    136.532992   
          Brand Group 41   2427.396563   3221.070147   3093.632493   
          Brand Group 43      0.000000      0.000000      0.000000   
          Brand Group 44    576.048693    871.090969   1361.650537   
          Brand Group 45   1995.503104   1663.363557   1307.480906   
          Brand Group 46   1335.533122   1394.229661   1505.275423   
          Brand Group 51    368.564296    400.115372    594.180684   
          Brand Group 58   1997.748581   1305.279925   1295.629396   
          Brand Group 63  -2408.708191    -69.668974    521.596040   
          Brand Group 70    553.832361   1916.631439    991.594098   
          Brand Group 71     40.007880     36.881432     74.881322   
          Brand Group 72    136.189910     68.737912    131.765769   
          Brand Group 73      0.000000      0.000000      0.000000   
...                                ...           ...           ...   
Cluster 9 Brand Group 17      0.000000      0.000000      0.000000   
          Brand Group 19   1065.970694    965.448115    826.204342   
          Brand Group 24      0.000000      0.000000      0.000000   
          Brand Group 25   1311.951469   1228.487730   1596.800828   
          Brand Group 26    578.979771    543.785442    501.128984   
          Brand Group 27    247.781298    204.107500    311.394846   
          Brand Group 30    849.946315    827.654128    931.822650   
          Brand Group 31     33.307956     37.928092     34.382406   
          Brand Group 36    411.032237    203.143857    326.407941   
          Brand Group 38     99.350845     45.914903     62.179995   
          Brand Group 39     49.729809     58.723182     58.563123   
          Brand Group 41  21390.272688  23210.413172  26803.499781   
          Brand Group 45    530.340734    465.930587    457.566091   
          Brand Group 46   2933.358538   2485.503539   3695.535976   
          Brand Group 51    109.127156    144.024336    126.068284   
          Brand Group 58    600.962268    495.003715    606.023913   
          Brand Group 70    123.079074    102.217856    118.692469   
          Brand Group 71    115.778040    101.534790    104.466731   
          Brand Group 72      1.273245      2.122076      1.697661   
          Brand Group 73      0.000000      0.000000      0.000000   
          Brand Group 76    395.554874    233.415453    362.856133   
          Brand Group 78     59.237114     50.176649     73.802040   
          Brand Group 79      1.564914      0.959141      1.682703   
          Brand Group 84   1223.608330   1416.356354   1882.021613   
          Brand Group 87   1350.142011  

In [721]:
dfpred=pd.melt(dfsales2.reset_index(), id_vars=["Cluster", "Brand Group"], 
                  var_name="Date", value_name="Value")

In [722]:
dfsales2.reset_index()

,Cluster,Brand Group,Jan 2012,Feb 2012,Mar 2012,Apr 2012,May 2012,Jun 2012,Jul 2012,Aug 2012,...,Mar 2017,Apr 2017,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017
0,Cluster 1,Brand Group 12,616.408961,685.081035,968.938870,723.701936,656.607506,319.401731,953.191022,658.304347,...,1494.614561,1583.525099,1396.117515,1471.551885,1632.173591,1627.574308,1608.715411,1538.061964,1574.399619,1.381174e+03
1,Cluster 1,Brand Group 13,62.381470,69.445132,42.081080,55.139918,65.677360,36.194606,46.670999,62.483972,...,5.496543,14.480741,9.765432,11.746420,7.310123,10.965185,5.803457,5.608148,8.537778,6.361481e+00
2,Cluster 1,Brand Group 15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,613.388915,-229.544362,798.197908,-193.611296,180.088396,-126.788616,35.634313,177.382970,48.785139,-2.653626e+02
3,Cluster 1,Brand Group 16,2457.695992,2616.171336,2019.736343,2521.239601,2341.879305,1909.803910,2335.718836,2284.694096,...,556.856275,516.432602,711.261828,575.720936,638.983822,590.373449,612.722539,535.401404,566.573007,5.391845e+02
4,Cluster 1,Brand Group 17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1632.612631,1990.658124,1586.519705,1959.033915,2334.987199,2371.470463,2283.761079,2830.252855,2530.420686,2.870711e+03
5,Cluster 1,Brand Group 18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.704348,-5.247172,0.447933,3.171981,-68.883897,40.824615,-0.139374,0.011042,-23.587434,3.298240e+02
6,Cluster 1,Brand Group 19,1328.072408,1747.087079,1843.272270,2039.346028,1691.095349,2047.555687,1610.354841,1770.380294,...,893.457850,855.506720,802.064436,1066.648259,785.950844,886.596334,755.373756,959.422827,953.348555,6.881483e+02
7,Cluster 1,Brand Group 24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,535.500112,617.733001,744.317609,906.058486,837.815300,754.027434,1193.776122,1224.923058,1434.480782,1.314431e+03
8,Cluster 1,Brand Group 25,1869.355444,1724.577575,3067.466622,2478.010511,1714.321134,1882.068631,2453.273551,2329.189122,...,1425.465837,1150.809671,1356.505256,1254.493173,1282.958362,1269.700506,1429.508574,1275.457380,1358.765291,1.249301e+03
9,Cluster 1,Brand Group 26,2044.415276,2124.060822,2773.263487,2272.917754,2528.122320,2218.761723,2335.513978,2267.608240,...,2842.389283,2684.698994,2723.677431,2740.300859,2542.729046,2499.681858,2571.601476,2224.735039,2913.654991,2.441371e+03


In [723]:
dfindexes=dfpred=dfsales2.reset_index()[["Cluster","Brand Group"]]

In [656]:
# GET LABELSSSSSSSSSSSSSSSs

In [657]:
dfpred=dfsales2.reset_index()[["Sep 2017","Oct 2017","Nov 2017","Dec 2017"]]

In [658]:
dfpred["Month"]=12

In [659]:
dfpred

,Sep 2017,Oct 2017,Nov 2017,Dec 2017,Month
0,1608.715411,1538.061964,1574.399619,1.381174e+03,12
1,5.803457,5.608148,8.537778,6.361481e+00,12
2,35.634313,177.382970,48.785139,-2.653626e+02,12
3,612.722539,535.401404,566.573007,5.391845e+02,12
4,2283.761079,2830.252855,2530.420686,2.870711e+03,12
5,-0.139374,0.011042,-23.587434,3.298240e+02,12
6,755.373756,959.422827,953.348555,6.881483e+02,12
7,1193.776122,1224.923058,1434.480782,1.314431e+03,12
8,1429.508574,1275.457380,1358.765291,1.249301e+03,12
9,2571.601476,2224.735039,2913.654991,2.441371e+03,12


In [660]:
dfpred.columns=["Lag-3","Lag-2","Lag-1","Actual","Month"]

In [661]:
dfpred['sales_trend'] = dfpred['Actual']

lag_list=[1,2,3]

for lag in [1,2,3]:
    ft_name = ('Lag-%s' % lag)
    dfpre['sales_trend'] -= dfpre[ft_name]

dfpre['sales_trend'] /= len(lag_list) + 1

In [662]:
dfpred.head(5)

,Lag-3,Lag-2,Lag-1,Actual,Month,sales_trend
0,1608.715411,1538.061964,1574.399619,1381.174249,12,1381.174249
1,5.803457,5.608148,8.537778,6.361481,12,6.361481
2,35.634313,177.382970,48.785139,-265.362597,12,-265.362597
3,612.722539,535.401404,566.573007,539.184516,12,539.184516
4,2283.761079,2830.252855,2530.420686,2870.710849,12,2870.710849


In [663]:
#dfpred[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']] =sc.transform(dfpred[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])

In [664]:
def add_values(pred_x, new_pred,month):
    pred_x['Month']=month
    
    pred_x["Lag-3"]=pred_x["Lag-2"]
    pred_x["Lag-2"]=pred_x["Lag-1"]
    pred_x["Lag-1"]=pred_x["Actual"]
    
    pred_x['sales_trend'] = pred_x['Actual']
    lag_list=[1,2,3]

    for lag in [1,2,3]:
        ft_name = ('Lag-%s' % lag)
        pred_x['sales_trend'] -= pred_x[ft_name]

    pred_x['sales_trend'] /= len(lag_list) + 1
    
    return pred_x

In [665]:
list_predictions=[]

In [666]:
xpred=dfpred[['Month', 'Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']]

In [667]:
for i in range(12):
    preds=model.predict(xpred)
    list_predictions.append(preds)
    xpred=add_values(xpred, preds,i)

In [668]:
len(list_predictions[0])

303

In [669]:
for d in range(len(list_predictions)):
    dfindexes['Month-%s' % d]=list_predictions[d]

In [670]:
dfindexes["Brand Group"].unique()

array(['Brand Group 12', 'Brand Group 13', 'Brand Group 15',
       'Brand Group 16', 'Brand Group 17', 'Brand Group 18',
       'Brand Group 19', 'Brand Group 24', 'Brand Group 25',
       'Brand Group 26', 'Brand Group 27', 'Brand Group 28',
       'Brand Group 30', 'Brand Group 31', 'Brand Group 36',
       'Brand Group 38', 'Brand Group 39', 'Brand Group 40',
       'Brand Group 41', 'Brand Group 43', 'Brand Group 44',
       'Brand Group 45', 'Brand Group 46', 'Brand Group 51',
       'Brand Group 58', 'Brand Group 63', 'Brand Group 70',
       'Brand Group 71', 'Brand Group 72', 'Brand Group 73',
       'Brand Group 76', 'Brand Group 78', 'Brand Group 79',
       'Brand Group 80', 'Brand Group 84', 'Brand Group 85',
       'Brand Group 87', 'Brand Group 89', 'Brand Group 90',
       'Brand Group 92', 'Brand Group 94', 'Brand Group 96',
       'Brand Group 97', 'Brand Group 20', 'Brand Group 98',
       'Brand Group 60', 'Brand Group 64', 'Brand Group 21'], dtype=object)

In [671]:
dfindexes

,Cluster,Brand Group,Month-0,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11
0,Cluster 1,Brand Group 12,1632.197632,1211.584351,1377.070679,1342.979492,1334.370850,1314.214111,1318.171875,1292.714355,1344.660767,1330.777466,1337.252075,1358.666138
1,Cluster 1,Brand Group 13,81.828377,5.216658,7.284895,7.227825,9.626293,6.860891,6.082820,5.639297,5.837836,3.032424,7.160820,9.652314
2,Cluster 1,Brand Group 15,-17.568844,-152.548080,-160.863312,-225.174866,-225.174866,-225.796204,-225.796204,-236.823608,-236.823608,-219.102570,-207.074249,-208.657043
3,Cluster 1,Brand Group 16,716.253052,497.814423,532.475098,572.096741,580.056580,582.576111,586.883728,505.316315,540.771851,578.422729,589.919495,580.464600
4,Cluster 1,Brand Group 17,2718.486084,2671.920166,2842.251221,2815.820068,2832.056885,2805.912842,2999.287842,2625.524414,2763.027832,2768.531006,2758.083740,2734.190674
5,Cluster 1,Brand Group 18,477.937714,200.379440,330.634918,323.064392,329.316345,334.898804,343.011444,286.582672,310.745758,319.004303,318.330994,329.414490
6,Cluster 1,Brand Group 19,1379.575562,745.721375,692.558777,660.516602,665.254578,671.182861,677.866089,664.225708,673.617432,683.509705,683.509705,676.470154
7,Cluster 1,Brand Group 24,1553.715942,1191.722290,1359.901611,1384.113281,1387.187500,1377.210449,1345.177979,1272.497070,1381.827881,1383.881226,1372.453857,1370.119263
8,Cluster 1,Brand Group 25,1569.044922,1171.419434,1240.051880,1177.973389,1181.991577,1181.991577,1208.736328,1113.591553,1216.955933,1202.696411,1192.226440,1227.549927
9,Cluster 1,Brand Group 26,2598.563721,2372.460449,2551.495361,2505.745850,2470.091064,2481.990234,2621.901367,2249.249268,2370.468750,2438.719482,2453.526123,2476.946045


In [672]:
dfindexes.iloc[0][2:]

Month-0      1632.2
Month-1     1211.58
Month-2     1377.07
Month-3     1342.98
Month-4     1334.37
Month-5     1314.21
Month-6     1318.17
Month-7     1292.71
Month-8     1344.66
Month-9     1330.78
Month-10    1337.25
Month-11    1358.67
Name: 0, dtype: object

In [673]:
dir_submission_template = 'Data_Novartis_Datathon-Results_Challenge1_Template.csv'

submission_template = pd.read_csv(dir_submission_template)

bg_set = set()

#a = pd.DataFrame(final_res, columns=['Cluster', 'Brand Group', 'res'])

for i, r in submission_template.iterrows():
    bg = r['Brand Group'][12:].replace(',', '').split(' ')
    bg_set.update(bg)
    bg = [ 'Brand Group ' + str(i) for i in bg ]
    print(r['Cluster'], r['Brand Group'], bg)
    
#    print(type(r['res']))
    
    res = dfindexes[dfindexes['Cluster'] == r['Cluster']]
    res = [ rs[2:] for j, rs in res.iterrows() if rs['Brand Group'] in bg ]
#    res = res['res']
    res = np.sum(res, axis=0)
    submission_template.iloc[i, 2:] = res
    
    
for i, r in submission_template[submission_template['Brand Group'] == 'others'].iterrows():
    print(r.Cluster)
    
    res = dfindexes[dfindexes['Cluster'] == r['Cluster']]
    res1 = [ rs[2:] for j, rs in res.iterrows() if rs['Brand Group'][12:] not in bg_set ]

    res2 = np.sum(res1, axis=0)
    submission_template.iloc[i, 2:] = res2

Cluster 1 Brand Group 17 ['Brand Group 17']
Cluster 2 Brand Group 17 ['Brand Group 17']
Cluster 3 Brand Group 17 ['Brand Group 17']
Cluster 4 Brand Group 17 ['Brand Group 17']
Cluster 5 Brand Group 17 ['Brand Group 17']
Cluster 8 Brand Group 17 ['Brand Group 17']
Cluster 9 Brand Group 17 ['Brand Group 17']
Cluster 10 Brand Group 17 ['Brand Group 17']
Cluster 1 Brand Group 24 ['Brand Group 24']
Cluster 3 Brand Group 24 ['Brand Group 24']
Cluster 4 Brand Group 24 ['Brand Group 24']
Cluster 5 Brand Group 24 ['Brand Group 24']
Cluster 8 Brand Group 24 ['Brand Group 24']
Cluster 9 Brand Group 24 ['Brand Group 24']
Cluster 10 Brand Group 24 ['Brand Group 24']
Cluster 1 Brand Group 30 ['Brand Group 30']
Cluster 2 Brand Group 30 ['Brand Group 30']
Cluster 4 Brand Group 30 ['Brand Group 30']
Cluster 5 Brand Group 30 ['Brand Group 30']
Cluster 7 Brand Group 30 ['Brand Group 30']
Cluster 8 Brand Group 30 ['Brand Group 30']
Cluster 9 Brand Group 30 ['Brand Group 30']
Cluster 10 Brand Group 30 ['Br

In [675]:
submission_template.to_csv("submission_2.csv",index=False)